In [ ]:
import torch
import torch.nn as nn
import numpy as np
from models import GKT




In [ ]:
# 하이퍼파라미터 및 모델 초기화
concept_num = 50     # 문항 개수 (문제 수)
hidden_dim = 128     # 은닉 상태 크기
embedding_dim = 64   # 임베딩 차원
edge_type_num = 2    # 그래프 엣지 타입 개수
graph_type = 'Dense' # 그래프 타입 (Dense, Transition 등)
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# 모델 초기화
model = GKT(concept_num=concept_num, hidden_dim=hidden_dim, embedding_dim=embedding_dim,
            edge_type_num=edge_type_num, graph_type=graph_type, has_cuda=torch.cuda.is_available())

In [ ]:
# 모델 가중치 로드 (.pt 파일)
model_path = "./logs/expGKT-MHA2024-12-17 11-38-53/GKT_MHA.pt"  # 저장된 모델 파일 경로
state_dict = torch.load(model_path, map_location=device)
model.load_state_dict(state_dict)  # 가중치 적용
model.to(device)
model.eval()  # 평가 모드 설정

In [ ]:
# 예제 입력 데이터
batch_size = 2
seq_len = 10
features = torch.randint(0, 2, (batch_size, seq_len)).to(device)  # [batch_size, seq_len], 입력 특성
questions = torch.randint(0, concept_num, (batch_size, seq_len)).to(device)  # [batch_size, seq_len], 문제 인덱스


In [ ]:
# 예측값 계산
with torch.no_grad():
    pred_res, _, _, _ = model(features, questions)  # 예측값 반환